# Segmentación de Clientes mediante K-Means

## Objetivo

El objetivo de este análisis es identificar segmentos homogéneos de clientes a partir de sus comportamientos de navegación, compra y características sociodemográficas. Esta segmentación permitirá mejorar las estrategias de marketing, personalizar las recomendaciones y optimizar las campañas de retención y adquisición.

## Dataset

Partimos de un dataset limpio que consolida información transaccional, eventos de navegación y metadatos de producto. Las variables disponibles incluyen:

- **Identificadores**: `customer_id`, `event_id`, `session_id`, `product_id`, `booking_id`
- **Datos geográficos**: `home_location_lat`, `home_location_long`, `home_location`, `shipment_location_lat`, `shipment_location_long`
- **Comportamiento temporal**: `first_join_date`, `event_time`, `transaction_time`, `shipment_date_limit`
- **Atributos de navegación y compra**: `event_name`, `traffic_source`, `device_type`, `search_keywords`, `was_purchased`
- **Información transaccional**: `item_price`, `product_quantity`, `total_amount`, `shipment_fee`, `promo_code`, `promo_amount`, `payment_method`, `payment_status`
- **Características del cliente**: `customer_gender`, `gender`
- **Metadatos del producto**: `mastercategory`, `subcategory`, `articletype`, `basecolour`, `season`, `year`, `usage`, `productdisplayname`

## Enfoque

El proceso de segmentación se compone de las siguientes etapas:

1. **Selección de variables relevantes** para representar el comportamiento y valor de los clientes.
2. **Agregación y transformación de datos** a nivel cliente (`customer_id`) para consolidar las métricas.
3. **Preprocesamiento**, incluyendo normalización y codificación.
4. **Determinación del número óptimo de clusters** mediante técnicas como el método del codo y el coeficiente de Silhouette.
5. **Aplicación del algoritmo K-Means** para asignar cada cliente a un segmento.
6. **Análisis e interpretación** de los segmentos resultantes mediante visualizaciones y estadísticas descriptivas.


## Variables Seleccionadas para la Segmentación de Clientes

A continuación se detallan las variables agregadas a nivel cliente (`customer_id`) que se utilizarán como base para aplicar el algoritmo K-Means.

### Comportamiento de Compra
- **`total_amount_sum`**: Importe total gastado por el cliente en sus compras.
- **`product_quantity_sum`**: Número total de productos adquiridos.
- **`num_purchases`**: Número de eventos donde se produjo una compra (`was_purchased = 1`).
- **`avg_order_value`**: Valor medio de cada compra realizada.

### Variables Temporales
- **`recency_days`**: Número de días desde la última compra hasta la fecha de referencia.
- **`first_purchase_days`**: Días desde la primera compra registrada (antigüedad como cliente).
- **`average_time_transaction`**: Tiempo promedio que el cliente permanece en una sesión con transacción.

### Interacción Digital
- **`num_sessions`**: Total de sesiones únicas iniciadas por el cliente.
- **`num_events`**: Número total de eventos (interacciones) registrados.
- **`distinct_product_views`**: Número de productos diferentes visualizados.
- **`num_searches`**: Número de búsquedas realizadas por el cliente (`search_keywords` no nulo).

### Promociones y Métodos de Pago
- **`promo_usage_rate`**: Proporción de compras en las que se aplicó una promoción.
- **`avg_promo_discount`**: Descuento medio aplicado en las compras con promoción.
- **`most_used_payment_method`**: Método de pago más frecuentemente utilizado.

### Perfil del Cliente
- **`customer_gender`**: Género del cliente.
- **`birthdate`**:Fecha de nacimiento del cliente. La vamos a convertir en edad.
- **`device_type_mode`**: Tipo de dispositivo más utilizado en las sesiones.
- **`traffic_source_mode`**: Fuente de tráfico más común por la que accede el cliente.
- **`home_location`**: Localización geográfica de residencia del cliente (puede codificarse o agruparse).
- **`home_location_long`, `home_location_lat`**: coordenadas del domicilio del cliente.
> **Nota**: Las variables categóricas deben ser codificadas (por ejemplo, con one-hot encoding), y las numéricas normalizadas antes de aplicar K-Means.


In [1]:
import pandas as pd
from sqlalchemy import create_engine,text
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import shapiro
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import geopandas as gpd
import contextily as cx

In [2]:
# Cargar variables de entorno
load_dotenv(override=True)

# Configuración de conexión
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_PORT = os.getenv("DB_PORT")

# Crear el motor de conexión a PostgreSQL
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

### **Tabla Global para la Segmentación de Clientes: customer_segmentation_features**

Para llevar a cabo el proceso de **Segmentación de Clientes mediante K-Means**, se ha diseñado una tabla global llamada `customer_segmentation_features` que servirá como repositorio único para consolidar todas las variables seleccionadas.

---

### **Estructura de la Tabla Global**
La tabla `customer_segmentation_features` está compuesta por un identificador único para cada cliente (`customer_id`) y un conjunto de columnas que representan distintos aspectos del comportamiento del cliente. Estas columnas se irán añadiendo de manera **incremental y optimizada**, asegurando que cada actualización sea eficiente y no genere bloqueos en la base de datos.

---

### **Proceso de Incrementación de Variables**
El proceso para añadir cada variable a la tabla global sigue los siguientes pasos:

**Validación y Creación de la Tabla:**  
Si la tabla no existe, se crea con los campos necesarios. En un primer momento, solo se define el identificador (`customer_id`).  

**Cálculo de la Variable:**  
Se realiza un cálculo agregado sobre la tabla principal (`cleaned_base_table`) para obtener el valor correspondiente para cada cliente.  

**Inserción o Actualización:**  
- Si el cliente ya existe en la tabla, se actualiza su valor para esa variable.  
- Si el cliente no existe, se inserta un nuevo registro con ese valor.  

**Verificación de Consistencia:**  
Después de cada incremento, se realiza una comprobación de datos para asegurar que se ha calculado correctamente.  

---

### **Variable 1: total_amount_sum**
Esta variable representa el **importe total gastado** por el cliente en todas sus transacciones registradas. Nos permite entender el valor monetario total que un cliente aporta al negocio.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se filtran únicamente las transacciones donde el producto fue efectivamente comprado (was_purchased = TRUE).
- Se suman los valores de la columna total_amount directamente por cliente, sin necesidad de agrupar por sesión.
- Posteriormente, se agrupa por `customer_id` para obtener el gasto total acumulado del cliente.



In [6]:
%%time
# SQL para crear la tabla y calcular total_amount_sum
sql = """
DROP TABLE IF EXISTS customer_segmentation_features;

CREATE TABLE customer_segmentation_features AS
SELECT 
    c.customer_id,
    COALESCE(SUM(t.total_amount_final), 0) AS total_amount_sum
FROM (
    SELECT DISTINCT customer_id FROM cleaned_base_table
    WHERE customer_id IS NOT NULL
) c
LEFT JOIN (
    SELECT 
        customer_id, 
        SUM(total_amount_final) AS total_amount_final
    FROM cleaned_base_table
    WHERE was_purchased = TRUE AND customer_id IS NOT NULL
    GROUP BY customer_id
) t ON c.customer_id = t.customer_id
GROUP BY c.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando creación de 'customer_segmentation_features'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Tabla creada correctamente con todos los clientes.")


Ejecutando creación de 'customer_segmentation_features'...
✅ Tabla creada correctamente con todos los clientes.
CPU times: total: 15.6 ms
Wall time: 1min 51s


### **Variable 2: product_quantity_sum**
Esta variable representa el **número total de productos adquiridos** por el cliente en todas sus transacciones registradas. Nos permite entender el volumen de artículos que un cliente ha comprado a lo largo del tiempo.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se filtran únicamente las transacciones donde el producto fue efectivamente comprado (`was_purchased = TRUE`).
- Se suman los valores de la columna `product_quantity_winsorized` directamente por cliente, sin necesidad de agrupar por sesión.
- El resultado es la suma total de artículos adquiridos por cada cliente.


In [7]:
%%time
# SQL para agregar la columna product_quantity_sum en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN product_quantity_sum FLOAT;

UPDATE customer_segmentation_features cs
SET product_quantity_sum = COALESCE(sub.total_quantity, 0)
FROM (
    SELECT 
        customer_id,
        SUM(product_quantity_winsorized) AS total_quantity
    FROM cleaned_base_table
    WHERE was_purchased = TRUE AND customer_id IS NOT NULL
    GROUP BY customer_id
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando actualización para 'product_quantity_sum'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'product_quantity_sum' agregada correctamente.")


Ejecutando actualización para 'product_quantity_sum'...
✅ Columna 'product_quantity_sum' agregada correctamente.
CPU times: total: 0 ns
Wall time: 1min 8s


### **Variable 3: num_purchases**
Esta variable representa el **número total de compras realizadas** por el cliente. Nos permite identificar la frecuencia con la que un cliente adquiere productos en la plataforma.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se contabilizan todas las sesiones (`session_id`) donde al menos un producto fue comprado (`was_purchased = TRUE`).
- Se agrupa por `customer_id` para obtener el número total de sesiones de compra.
- El valor final representa la cantidad de ocasiones en que el cliente ha realizado una compra efectiva.



In [8]:
%%time
# SQL para agregar la columna num_purchases en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN num_purchases INT;

UPDATE customer_segmentation_features cs
SET num_purchases = COALESCE(sub.total_purchases, 0)
FROM (
    SELECT 
        customer_id,
        COUNT(DISTINCT session_id) AS total_purchases
    FROM cleaned_base_table
    WHERE was_purchased = TRUE AND customer_id IS NOT NULL
    GROUP BY customer_id
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando actualización para 'num_purchases'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'num_purchases' agregada correctamente.")

Ejecutando actualización para 'num_purchases'...
✅ Columna 'num_purchases' agregada correctamente.
CPU times: total: 0 ns
Wall time: 1min 4s


### **Variable 4: avg_order_value**
Esta variable representa el **valor promedio de cada compra realizada** por el cliente. Nos permite entender el ticket medio de los clientes en cada transacción.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se toma el valor total gastado por el cliente (`total_amount_sum`).
- Se divide por el número de compras realizadas (`num_purchases`).
- En caso de que el número de compras sea 0, el valor se establece en 0 para evitar divisiones por cero.


In [9]:
%%time
# SQL para agregar la columna avg_order_value en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN avg_order_value FLOAT;

UPDATE customer_segmentation_features
SET avg_order_value = 
    CASE 
        WHEN num_purchases > 0 THEN total_amount_sum / num_purchases 
        ELSE 0 
    END;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("🚀 Ejecutando actualización para 'avg_order_value'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'avg_order_value' agregada correctamente.")

🚀 Ejecutando actualización para 'avg_order_value'...
✅ Columna 'avg_order_value' agregada correctamente.
CPU times: total: 0 ns
Wall time: 236 ms


### **Variable 5: recency_days**
Esta variable representa el **número de días transcurridos desde la última compra registrada hasta la fecha actual**. Nos permite medir el tiempo de inactividad del cliente y evaluar su nivel de reciente actividad.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se obtiene la fecha más reciente en la que el cliente realizó una transacción (`transaction_time`).
- Se calcula la diferencia en días entre esa fecha y el día actual.
- Si el cliente nunca ha comprado, se asigna un valor nulo (o un valor muy alto que indique inactividad total).



In [10]:
%%time
# SQL para agregar la columna recency_days en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN recency_days INT;

UPDATE customer_segmentation_features cs
SET recency_days = COALESCE(sub.days_since_last_purchase, 9999)
FROM (
    SELECT 
        customer_id,
        DATE_PART('day', CURRENT_DATE - MAX(transaction_time)) AS days_since_last_purchase
    FROM cleaned_base_table
    WHERE customer_id IS NOT NULL AND was_purchased = TRUE
    GROUP BY customer_id
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando actualización para 'recency_days'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'recency_days' agregada correctamente.")


Ejecutando actualización para 'recency_days'...
✅ Columna 'recency_days' agregada correctamente.
CPU times: total: 0 ns
Wall time: 1min 3s


### **Variable 6: first_purchase_days**
Esta variable representa el **número de días transcurridos desde el primer registro del cliente hasta la fecha actual**. Nos permite identificar la antigüedad del cliente en la plataforma.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se obtiene la fecha en la que el cliente realizó su primer registro (`first_join_date`).
- Se calcula la diferencia en días entre esa fecha y el día actual.
- En caso de que el cliente no tenga una fecha de registro válida, se asigna un valor `NULL`.



In [11]:
%%time
# SQL para agregar la columna first_purchase_days en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN first_purchase_days INT;

UPDATE customer_segmentation_features cs
SET first_purchase_days = sub.days_since_first_join
FROM (
    SELECT 
        customer_id,
        DATE_PART('day', CURRENT_DATE::TIMESTAMP - first_join_date::TIMESTAMP) AS days_since_first_join
    FROM cleaned_base_table
    WHERE customer_id IS NOT NULL AND first_join_date IS NOT NULL
    GROUP BY customer_id, first_join_date
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("🚀 Ejecutando actualización para 'first_purchase_days'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'first_purchase_days' agregada correctamente.")


🚀 Ejecutando actualización para 'first_purchase_days'...
✅ Columna 'first_purchase_days' agregada correctamente.
CPU times: total: 15.6 ms
Wall time: 1min 5s


### **Variable 7: average_time_transaction**
Esta variable representa el **tiempo promedio que el cliente permanece en una sesión en la que se produce una transacción**. Nos permite entender el nivel de compromiso del usuario durante el proceso de compra.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se filtran las sesiones.
- Se calcula la diferencia de tiempo entre el primer evento y el último evento dentro de esa sesión.
- Posteriormente, se obtiene el promedio de tiempo para cada cliente.
- El valor se expresa en minutos.



In [12]:
%%time
# SQL para agregar la columna average_time_transaction en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN average_time_transaction FLOAT;

UPDATE customer_segmentation_features cs
SET average_time_transaction = COALESCE(sub.avg_session_time, 0)
FROM (
    SELECT 
        customer_id,
        AVG(session_duration) AS avg_session_time
    FROM (
        SELECT 
            customer_id,
            EXTRACT(EPOCH FROM (MAX(event_time) - MIN(event_time))) / 60.0 AS session_duration
        FROM cleaned_base_table
        WHERE customer_id IS NOT NULL 
    GROUP BY customer_id, session_id
    ) sessions
    GROUP BY customer_id
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando actualización para 'average_time_transaction'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'average_time_transaction' agregada correctamente.")


Ejecutando actualización para 'average_time_transaction'...
✅ Columna 'average_time_transaction' agregada correctamente.
CPU times: total: 0 ns
Wall time: 1min 9s


### **Variable 8: num_sessions**
Esta variable representa el **número total de sesiones únicas** iniciadas por el cliente en la plataforma. Nos permite identificar la frecuencia con la que un cliente navega por el sitio web o aplicación.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se cuentan todas las sesiones únicas (`session_id`) asociadas a cada cliente.
- No se filtra por `was_purchased`, ya que una sesión no necesariamente tiene que terminar en una compra para ser registrada.
- El resultado se guarda a nivel de cliente.



In [13]:
%%time
# SQL para agregar la columna num_sessions en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN num_sessions INT;

UPDATE customer_segmentation_features cs
SET num_sessions = COALESCE(sub.total_sessions, 0)
FROM (
    SELECT 
        customer_id,
        COUNT(DISTINCT session_id) AS total_sessions
    FROM cleaned_base_table
    WHERE customer_id IS NOT NULL
    GROUP BY customer_id
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando actualización para 'num_sessions'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'num_sessions' agregada correctamente.")


Ejecutando actualización para 'num_sessions'...
✅ Columna 'num_sessions' agregada correctamente.
CPU times: total: 0 ns
Wall time: 1min 10s


### **Variable 9: num_events**
Esta variable representa el **número total de eventos registrados** por el cliente en la plataforma. Un evento puede ser una visualización de producto, una búsqueda, un clic, un pago, entre otros. Nos permite entender el nivel de interacción y actividad del usuario.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se cuentan todos los eventos (`event_id`) asociados al cliente.
- No se filtra por tipo de evento, ya que todos los eventos representan una interacción.
- El resultado se guarda a nivel de cliente en la tabla global de segmentación.



In [14]:
%%time
# SQL para agregar la columna num_events en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN num_events INT;

UPDATE customer_segmentation_features cs
SET num_events = COALESCE(sub.total_events, 0)
FROM (
    SELECT 
        customer_id,
        COUNT(event_id) AS total_events
    FROM cleaned_base_table
    WHERE customer_id IS NOT NULL
    GROUP BY customer_id
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando actualización para 'num_events'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'num_events' agregada correctamente.")


Ejecutando actualización para 'num_events'...
✅ Columna 'num_events' agregada correctamente.
CPU times: total: 0 ns
Wall time: 1min 8s


### **Variable 10: distinct_product_views**
Esta variable representa el **número de productos únicos visualizados** por el cliente en la plataforma. Nos permite entender la amplitud de exploración del cliente en cuanto a productos y categorías.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se cuentan los productos únicos (`product_id`) que el cliente ha visualizado en sus sesiones.
- No se filtra por `was_purchased`, ya que una visualización no siempre termina en una compra.
- El valor se guarda a nivel de cliente en la tabla global de segmentación.



In [15]:
%%time
# SQL para agregar la columna distinct_product_views en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN distinct_product_views INT;

UPDATE customer_segmentation_features cs
SET distinct_product_views = COALESCE(sub.total_distinct_products, 0)
FROM (
    SELECT 
        customer_id,
        COUNT(DISTINCT product_id) AS total_distinct_products
    FROM cleaned_base_table
    WHERE customer_id IS NOT NULL AND product_id IS NOT NULL
    GROUP BY customer_id
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando actualización para 'distinct_product_views'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'distinct_product_views' agregada correctamente.")


Ejecutando actualización para 'distinct_product_views'...
✅ Columna 'distinct_product_views' agregada correctamente.
CPU times: total: 0 ns
Wall time: 1min 8s


### **Variable 11: num_searches**
Esta variable representa el **número total de búsquedas realizadas** por el cliente en la plataforma. Nos permite entender el interés del usuario en explorar productos o categorías específicas.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se cuentan las interacciones donde el campo `search_keywords` no es nulo.
- Cada búsqueda registrada se contabiliza para el cliente correspondiente.
- El valor se guarda a nivel de cliente en la tabla global de segmentación.


In [16]:
%%time
# SQL para agregar la columna num_searches en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN num_searches INT;

UPDATE customer_segmentation_features cs
SET num_searches = COALESCE(sub.total_searches, 0)
FROM (
    SELECT 
        customer_id,
        COUNT(*) AS total_searches
    FROM cleaned_base_table
    WHERE customer_id IS NOT NULL AND search_keywords IS NOT NULL
    GROUP BY customer_id
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando actualización para 'num_searches'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'num_searches' agregada correctamente.")


Ejecutando actualización para 'num_searches'...
✅ Columna 'num_searches' agregada correctamente.
CPU times: total: 0 ns
Wall time: 1min 3s


### **Variable 12: promo_usage_rate**
Esta variable representa la **proporción de compras en las que se utilizó un código promocional**. Nos permite identificar qué tan frecuente es el uso de promociones en las transacciones del cliente.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se cuentan todas las compras realizadas (`was_purchased = TRUE`).
- De esas compras, se filtran aquellas en las que se aplicó un código promocional (`has_promo = TRUE`).
- La métrica se calcula como el porcentaje de artículos con promoción respecto al total de artículos.
- Si el cliente no tiene compras, el valor se establece en `0`.


In [23]:
%%time
# SQL para agregar la columna promo_usage_rate en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN promo_usage_rate FLOAT;

UPDATE customer_segmentation_features cs
SET promo_usage_rate = COALESCE(sub.promo_quantity_sum, 0) / NULLIF(cs.product_quantity_sum, 0)
FROM (
    SELECT 
        customer_id,
        SUM(product_quantity) FILTER (WHERE has_promo = TRUE AND was_purchased = TRUE) AS promo_quantity_sum
    FROM cleaned_base_table
    WHERE customer_id IS NOT NULL
    GROUP BY customer_id
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("🚀 Ejecutando actualización para 'promo_usage_rate'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'promo_usage_rate' agregada correctamente.")


🚀 Ejecutando actualización para 'promo_usage_rate'...
✅ Columna 'promo_usage_rate' agregada correctamente.
CPU times: total: 0 ns
Wall time: 1min 10s


### **Variable 13: avg_promo_discount**
Esta variable representa el **descuento medio aplicado** en las compras en las que se utilizó un código promocional. Nos permite entender el nivel de descuento promedio que los clientes aprovechan en sus transacciones.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se filtran únicamente las compras en las que se aplicó un código promocional (`transaction_promo_amount_winsorized > 0`).
- Se calcula el promedio de esos descuentos para cada cliente.
- Si el cliente no ha utilizado promociones, el valor se establece en `0`.


In [24]:
%%time
# SQL para agregar la columna avg_promo_discount en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN avg_promo_discount FLOAT;

UPDATE customer_segmentation_features cs
SET avg_promo_discount = COALESCE(sub.average_discount, 0)
FROM (
    SELECT 
        customer_id,
        AVG(transaction_promo_amount_winsorized) AS average_discount
    FROM cleaned_base_table
    WHERE customer_id IS NOT NULL AND transaction_promo_amount_winsorized > 0 AND has_promo = TRUE
    GROUP BY customer_id
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando actualización para 'avg_promo_discount'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'avg_promo_discount' agregada correctamente.")


Ejecutando actualización para 'avg_promo_discount'...
✅ Columna 'avg_promo_discount' agregada correctamente.
CPU times: total: 0 ns
Wall time: 1min 5s


### **Variable 14: most_used_payment_method**
Esta variable representa el **método de pago más frecuentemente utilizado** por el cliente en sus transacciones. Nos permite identificar las preferencias del usuario al momento de realizar una compra.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se contabiliza la frecuencia de cada método de pago por cliente.
- Se selecciona el método de pago con mayor número de transacciones.
- Si el cliente no ha realizado compras, el valor se deja en `NULL`.


In [25]:
%%time
# SQL para agregar la columna most_used_payment_method en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN most_used_payment_method VARCHAR;

UPDATE customer_segmentation_features cs
SET most_used_payment_method = sub.payment_method
FROM (
    SELECT DISTINCT ON (customer_id)
        customer_id,
        payment_method
    FROM (
        SELECT 
            customer_id,
            payment_method,
            COUNT(*) AS usage_count
        FROM cleaned_base_table
        WHERE customer_id IS NOT NULL AND payment_method IS NOT NULL
        GROUP BY customer_id, payment_method
        ORDER BY customer_id, usage_count DESC
    ) sub
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando actualización para 'most_used_payment_method'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'most_used_payment_method' agregada correctamente.")


Ejecutando actualización para 'most_used_payment_method'...
✅ Columna 'most_used_payment_method' agregada correctamente.
CPU times: total: 0 ns
Wall time: 1min 6s


### **Variable 15: device_type_mode**
Esta variable representa el **tipo de dispositivo más utilizado** por el cliente en sus interacciones con la plataforma. Nos permite entender si el cliente prefiere navegar y comprar desde un ordenador, un móvil o una tablet.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se contabiliza la frecuencia de uso de cada tipo de dispositivo (`device_type`) por cliente.
- Se selecciona el dispositivo con mayor número de interacciones.
- Si el cliente no tiene interacciones registradas, el valor se deja en `NULL`.


In [26]:
%%time
# SQL para agregar la columna device_type_mode en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN device_type_mode VARCHAR;

UPDATE customer_segmentation_features cs
SET device_type_mode = sub.device_type
FROM (
    SELECT DISTINCT ON (customer_id)
        customer_id,
        device_type
    FROM (
        SELECT 
            customer_id,
            device_type,
            COUNT(*) AS usage_count
        FROM cleaned_base_table
        WHERE customer_id IS NOT NULL AND device_type IS NOT NULL
        GROUP BY customer_id, device_type
        ORDER BY customer_id, usage_count DESC
    ) sub
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando actualización para 'device_type_mode'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'device_type_mode' agregada correctamente.")


Ejecutando actualización para 'device_type_mode'...
✅ Columna 'device_type_mode' agregada correctamente.
CPU times: total: 0 ns
Wall time: 1min 6s


### **Variable 16: traffic_source_mode**
Esta variable representa la **fuente de tráfico más utilizada** por el cliente para acceder a la plataforma. Nos permite identificar si el cliente llega a través de campañas publicitarias, tráfico directo, motores de búsqueda, redes sociales, entre otros.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se contabiliza la frecuencia de cada fuente de tráfico (`traffic_source`) por cliente.
- Se selecciona la fuente con mayor número de accesos.
- Si el cliente no tiene accesos registrados, el valor se deja en `NULL`.


In [27]:
%%time
# SQL para agregar la columna traffic_source_mode en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN traffic_source_mode VARCHAR;

UPDATE customer_segmentation_features cs
SET traffic_source_mode = sub.traffic_source
FROM (
    SELECT DISTINCT ON (customer_id)
        customer_id,
        traffic_source
    FROM (
        SELECT 
            customer_id,
            traffic_source,
            COUNT(*) AS usage_count
        FROM cleaned_base_table
        WHERE customer_id IS NOT NULL AND traffic_source IS NOT NULL
        GROUP BY customer_id, traffic_source
        ORDER BY customer_id, usage_count DESC
    ) sub
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando actualización para 'traffic_source_mode'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'traffic_source_mode' agregada correctamente.")


Ejecutando actualización para 'traffic_source_mode'...
✅ Columna 'traffic_source_mode' agregada correctamente.
CPU times: total: 15.6 ms
Wall time: 1min 6s


### **Variable 17: home_location**
Esta variable representa la **localización geográfica de residencia** del cliente. Nos permite entender la distribución geográfica de la base de clientes y facilita los análisis por regiones.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se selecciona el valor registrado en el campo `home_location` para cada cliente.
- El valor se actualiza en la tabla de segmentación.
- Si el cliente no tiene un valor de localización registrado, se deja en `NULL`.


In [28]:
%%time
# SQL para agregar la columna home_location en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN home_location VARCHAR;

UPDATE customer_segmentation_features cs
SET home_location = sub.home_location
FROM (
    SELECT DISTINCT ON (customer_id)
        customer_id,
        home_location
    FROM cleaned_base_table
    WHERE customer_id IS NOT NULL AND home_location IS NOT NULL
    ORDER BY customer_id
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando actualización para 'home_location'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'home_location' agregada correctamente.")


Ejecutando actualización para 'home_location'...
✅ Columna 'home_location' agregada correctamente.
CPU times: total: 0 ns
Wall time: 1min 9s


### **Variable 18: home_location_lat** y **Variable 19: home_location_long**
Estas variables representan las **coordenadas geográficas de latitud y longitud** del domicilio del cliente. Nos permiten realizar análisis geoespaciales y segmentación basada en proximidad geográfica.

---

#### **Criterio de cálculo**:
Para calcular estos valores correctamente:
- Se seleccionan los valores registrados en los campos `home_location_lat` y `home_location_long` para cada cliente.
- Los valores se actualizan en la tabla de segmentación.
- Si el cliente no tiene un valor registrado para la latitud o longitud, se deja en `NULL`.


In [29]:
%%time
# SQL para agregar las columnas home_location_lat y home_location_long en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN home_location_lat FLOAT,
    ADD COLUMN home_location_long FLOAT;

UPDATE customer_segmentation_features cs
SET 
    home_location_lat = sub.home_location_lat,
    home_location_long = sub.home_location_long
FROM (
    SELECT DISTINCT ON (customer_id)
        customer_id,
        home_location_lat,
        home_location_long
    FROM cleaned_base_table
    WHERE customer_id IS NOT NULL 
    ORDER BY customer_id
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando actualización para 'home_location_lat' y 'home_location_long'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columnas 'home_location_lat' y 'home_location_long' agregadas correctamente.")


Ejecutando actualización para 'home_location_lat' y 'home_location_long'...
✅ Columnas 'home_location_lat' y 'home_location_long' agregadas correctamente.
CPU times: total: 0 ns
Wall time: 1min 10s


### **Variable 20: customer_gender**
Esta variable representa el **género del cliente**. Nos permite segmentar a los clientes en grupos demográficos y analizar comportamientos diferenciados por género.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se selecciona el valor registrado en el campo `customer_gender` para cada cliente.
- El valor se actualiza en la tabla de segmentación.
- Si el cliente no tiene un valor registrado para género, se deja en `NULL`.


In [30]:
%%time
# SQL para agregar la columna customer_gender en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN customer_gender VARCHAR;

UPDATE customer_segmentation_features cs
SET customer_gender = sub.customer_gender
FROM (
    SELECT DISTINCT ON (customer_id)
        customer_id,
        customer_gender
    FROM cleaned_base_table
    WHERE customer_id IS NOT NULL AND customer_gender IS NOT NULL
    ORDER BY customer_id
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando actualización para 'customer_gender'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'customer_gender' agregada correctamente.")


Ejecutando actualización para 'customer_gender'...
✅ Columna 'customer_gender' agregada correctamente.
CPU times: total: 0 ns
Wall time: 1min 14s


### **Variable 21: age**
Esta variable representa la **edad actual del cliente**. Nos permite realizar segmentaciones etarias y entender la composición demográfica de los clientes.

---

#### **Criterio de cálculo**:
Para calcular este valor correctamente:
- Se resta la fecha de nacimiento (`birth_date`) de la fecha actual (`CURRENT_DATE`).
- El resultado se expresa en años completos.
- Si el cliente no tiene fecha de nacimiento registrada, se deja en `NULL`.


In [31]:
%%time
# SQL para agregar la columna age en la tabla global
sql = """
ALTER TABLE customer_segmentation_features
    ADD COLUMN age INT;

UPDATE customer_segmentation_features cs
SET age = sub.customer_age
FROM (
    SELECT DISTINCT ON (customer_id)
        customer_id,
        DATE_PART('year', AGE(CURRENT_DATE, birthdate::DATE)) AS customer_age
    FROM cleaned_base_table
    WHERE customer_id IS NOT NULL AND birthdate IS NOT NULL
    ORDER BY customer_id
) sub
WHERE cs.customer_id = sub.customer_id;
"""

# Ejecutar el SQL en la base de datos
with engine.connect() as connection:
    print("Ejecutando actualización para 'age'...")
    connection.execute(text(sql))
    connection.commit()
    print("✅ Columna 'age' agregada correctamente.")


Ejecutando actualización para 'age'...
✅ Columna 'age' agregada correctamente.
CPU times: total: 15.6 ms
Wall time: 1min 13s


# Preparación de Variables para Análisis

En este notebook hemos consolidado una tabla que contiene las variables necesarias para llevar a cabo el análisis posterior. Para ello, hemos realizado los cálculos oportunos en aquellas variables que requerían transformaciones o ajustes, garantizando así la calidad y consistencia de los datos.

Esta tabla servirá como base para el análisis de las variables en el próximo notebook, donde exploraremos en detalle la distribución y la relevancia de cada variable en el contexto del modelo de clustering.

Continuemos con el análisis en el siguiente notebook, donde profundizaremos en la interpretación de las variables seleccionadas.
